# TODIM Ranking

In [19]:
import math                      # for sqrt and other functions
import numpy as np               # for linear algebra
import pandas as pd              # for tabular output
from scipy.stats import rankdata # for ranking the candidates

## Step 0 - Obtaining and preprocessing data

In [20]:
bowlers_data = {
    'weights': '../data/bowling_criteria.csv',
    'scores': '../data/bowlers.csv',
}
batsmen_data = {
    'weights': '../data/batting_criteria.csv',
    'scores': '../data/batsmen.csv',
}
data = bowlers_data

In [21]:
attributes_data = pd.read_csv(data['weights'])
attributes_data

,Name,Ranking,Ideally
0,SR,1,Lower
1,Econ,2,Lower
2,Avg,3,Lower
3,Wkts,4,Higher
4,Runs,5,Lower
5,Inns,6,Higher
6,TBB,7,Higher
7,4w,8,Higher
8,Mat,9,Higher


In [22]:
benefit_attributes = set()
attributes = []
ranks = []
n = 0

for i, row in attributes_data.iterrows():
    attributes.append(row['Name'])
    ranks.append(float(row['Ranking']))
    n += 1
    
    if row['Ideally'] == 'Higher':
        benefit_attributes.add(i)

ranks = np.array(ranks)

In [23]:
weights = 2 * (n + 1 - ranks) / (n * (n + 1))
pd.DataFrame(data=weights, index=attributes, columns=['Weight'])

,Weight
SR,0.200000
Econ,0.177778
Avg,0.155556
Wkts,0.133333
Runs,0.111111
Inns,0.088889
TBB,0.066667
4w,0.044444
Mat,0.022222


In [24]:
original_dataframe = pd.read_csv(data['scores'])
candidates = original_dataframe['Name'].to_numpy()
raw_data = pd.DataFrame(original_dataframe, columns=attributes).to_numpy()

dimensions = raw_data.shape
m = dimensions[0]
n = dimensions[1]

pd.DataFrame(data=raw_data, index=candidates, columns=attributes)

,SR,Econ,Avg,Wkts,Runs,Inns,TBB,4w,Mat
Andre Russell,16.45,9.51,26.09,11.0,287.0,12.0,181.0,0.0,14.0
Ben Stokes,16.83,11.23,31.50,6.0,189.0,6.0,101.0,0.0,9.0
Chris Morris,15.23,9.27,23.54,13.0,306.0,9.0,198.0,0.0,9.0
Dwayne Bravo,22.45,8.02,30.00,11.0,330.0,12.0,247.0,0.0,12.0
Imran Tahir,14.85,6.70,16.58,26.0,431.0,17.0,386.0,2.0,17.0
Jofra Archer,23.45,6.77,26.45,11.0,291.0,11.0,258.0,0.0,11.0
Kagiso Rabada,11.28,7.83,14.72,25.0,368.0,12.0,282.0,2.0,12.0
Keemo Paul,18.11,8.72,26.33,9.0,237.0,8.0,163.0,0.0,8.0
Lasith Malinga,16.81,9.77,27.38,16.0,438.0,12.0,269.0,2.0,12.0
Moeen Ali,25.00,6.76,28.17,6.0,169.0,9.0,150.0,0.0,11.0


## Step 1 - Normalizing the Ratings And Weights

$$
P_{ij} = \begin{cases}
\frac{x_{ij} - m_j}{M_j - m_j} & \text{if } j \in J_1\\
\frac{M-j - x_{ij}}{M_j - m_j} & \text{if } j \in J_2
\end{cases}
$$

$$
w_{rc} = \frac{w_c}{w_r}
$$

and $w_r = \text{max}\left\{w_c | c = 1, 2, \ldots, n\right\}$

where $i = 1, 2, \ldots, m$ and $j = 1, 2, \ldots, n$.

In [25]:
for j in range(n):
    column = raw_data[:,j]
    max_val = np.max(column)
    min_val = np.min(column)
    denom = max_val - min_val
    if denom == 0:
        denom = max_val if max_val != 0 else 1
    if j in benefit_attributes:
        raw_data[:,j] = (raw_data[:,j] - min_val) / denom
    else:
        raw_data[:,j] = (max_val - raw_data[:,j]) / denom

pd.DataFrame(data=raw_data, index=candidates, columns=attributes)

,SR,Econ,Avg,Wkts,Runs,Inns,TBB,4w,Mat
Andre Russell,0.662533,0.347475,0.430931,0.285714,0.549091,0.583333,0.280702,0.0,0.750000
Ben Stokes,0.637728,0.000000,0.160160,0.047619,0.905455,0.083333,0.000000,0.0,0.333333
Chris Morris,0.742167,0.395960,0.558559,0.380952,0.480000,0.333333,0.340351,0.0,0.333333
Dwayne Bravo,0.270888,0.648485,0.235235,0.285714,0.392727,0.583333,0.512281,0.0,0.583333
Imran Tahir,0.766971,0.915152,0.906907,1.000000,0.025455,1.000000,1.000000,1.0,1.000000
Jofra Archer,0.205614,0.901010,0.412913,0.285714,0.534545,0.500000,0.550877,0.0,0.500000
Kagiso Rabada,1.000000,0.686869,1.000000,0.952381,0.254545,0.583333,0.635088,1.0,0.583333
Keemo Paul,0.554178,0.507071,0.418919,0.190476,0.730909,0.250000,0.217544,0.0,0.250000
Lasith Malinga,0.639034,0.294949,0.366366,0.523810,0.000000,0.583333,0.589474,1.0,0.583333
Moeen Ali,0.104439,0.903030,0.326827,0.047619,0.978182,0.333333,0.171930,0.0,0.500000


In [26]:
max_weight = max(weights)
weights /= max_weight

pd.DataFrame(data=weights, index=attributes, columns=['Weight'])

,Weight
SR,1.000000
Econ,0.888889
Avg,0.777778
Wkts,0.666667
Runs,0.555556
Inns,0.444444
TBB,0.333333
4w,0.222222
Mat,0.111111


## Step 2 - Calculating Dominance Degrees

For the contribution of each criteria, we have:

$$
\Phi_{c}\left(A_i, A_j\right) = \begin{cases}
\sqrt{\frac{\left(P_{ic} - P_{jc}\right) w_{rc}}{\sum^{n}_{c=1} {w_{rc}}}} & \text{if } P_{ic} - P_{jc} > 0  \\
0 & \text{if } P_{ic} - P_{jc} = 0 \\
-\frac{1}{\theta}\sqrt{\frac{\left(\sum^{n}_{c=1} {w_{rc}}\right) \left(P_{jc} - P_{ic}\right)}{w_{rc}}} & \text{if } P_{ic} - P_{jc} < 0
\end{cases}
$$

Combining all contributions, we get the dominance degrees:

$$
\delta\left(A_i, A_j\right) = \sum^{n}_{c = 1} {\Phi_{c}\left(A_i, A_j\right)}
$$

Here $c = 1, 2, \ldots, n$, $i, j = 1, 2, \ldots, m$.

In [27]:
# The loss attenuation factor
theta = 1.0

In [28]:
phi = np.zeros((n, m, m))

weight_sum = sum(weights)

for c in range(n):
    for i in range(m):
        for j in range(m):
            pic = raw_data[i,c]
            pjc = raw_data[j,c]
            val = 0
            if pic > pjc:
                val = math.sqrt((pic - pjc) * weights[c] / weight_sum)
            if pic < pjc:
                val = -1.0 / theta * math.sqrt(weight_sum * (pjc - pic) / weights[c])
            phi[c, i, j] = val

In [29]:
delta = np.zeros((m, m))
for i in range(m):
    for j in range(m):
        delta[i,j] = sum(phi[:,i,j])

pd.DataFrame(data=delta, index=candidates, columns=candidates)

,Andre Russell,Ben Stokes,Chris Morris,Dwayne Bravo,Imran Tahir,Jofra Archer,Kagiso Rabada,Keemo Paul,Lasith Malinga,Moeen Ali,Mohammad Nabi,Rashid Khan,Sam Curran,Sunil Narine,Trent Boult
Andre Russell,0.000000,-0.644664,-3.517182,-2.517990,-19.879475,-3.221622,-13.636662,-1.581134,-7.658569,-2.784632,-6.951643,-10.745048,-3.325136,-2.577644,-1.852279
Ben Stokes,-12.960672,0.000000,-9.115638,-11.929223,-24.928594,-12.143522,-21.526759,-6.877878,-17.535634,-9.792911,-11.005991,-16.911790,-9.256557,-11.065912,-2.861416
Chris Morris,-6.260040,-0.787498,0.000000,-7.086455,-21.592397,-7.680325,-17.888329,-1.573063,-12.061288,-5.681163,-6.244502,-13.287054,-2.491143,-7.255411,-1.845118
Dwayne Bravo,-6.090309,-2.406541,-4.165624,0.000000,-20.933551,-3.907864,-12.804096,-3.351787,-8.971062,-3.556005,-7.977040,-12.432105,-3.966432,-0.878049,-1.273013
Imran Tahir,-0.431179,-0.676767,-0.342604,0.026284,0.000000,-0.469413,-2.563137,-0.613532,1.541252,-1.066441,-1.379484,-0.813542,0.099795,0.296433,-0.750983
Jofra Archer,-6.088082,-2.075328,-2.772154,-2.921642,-20.270227,0.000000,-14.572310,-2.096038,-10.560410,-1.387977,-6.561305,-12.182963,-3.408364,-2.672181,-1.338594
Kagiso Rabada,-2.901150,-0.391045,0.142709,0.293643,-10.069807,-1.271188,0.000000,-0.337947,1.308964,-1.981905,-2.005901,-7.287464,0.576684,0.593203,-0.558193
Keemo Paul,-9.201754,-2.955162,-7.066663,-9.048232,-23.385333,-9.158661,-19.647352,0.000000,-14.577795,-6.693789,-7.385360,-14.978181,-7.467130,-8.942058,-0.996666
Lasith Malinga,-5.959747,-1.484029,-3.960567,-2.415330,-15.876861,-3.723529,-8.980055,-3.270788,0.000000,-3.589388,-4.674126,-12.065455,-0.823191,-2.181796,-2.674566
Moeen Ali,-9.600713,-0.664355,-5.580414,-7.534467,-22.109323,-6.303964,-17.314454,-3.539589,-14.230034,0.000000,-7.570938,-14.504796,-6.622009,-6.445442,-0.400117


## Step 3 - Calculate ratings from the normalised dominance degree values

$$
\zeta_i = \frac{\sum_{j=1}^m {\delta\left(A_i, A_j\right)} - \delta_{\text{min}}}{\delta_{\text{max}} - \delta_{\text{min}}}
$$

where

$$
\delta_{\text{min}} = \min_i{\sum_{j=1}^m \delta\left(A_i, A_j\right)}
$$

$$
\delta_{\text{max}} = \max_i{\sum_{j=1}^m \delta\left(A_i, A_j\right)}
$$

and $i, j = 1, 2, \ldots, m$

In [30]:
delta_sums = np.zeros(m)
for i in range(m):
    delta_sums[i] = sum(delta[i,:])
pd.DataFrame(data=delta_sums,index=candidates,columns=['Sum'])

,Sum
Andre Russell,-80.893679
Ben Stokes,-177.912497
Chris Morris,-111.733786
Dwayne Bravo,-92.713479
Imran Tahir,-7.143317
Jofra Archer,-88.907575
Kagiso Rabada,-23.889398
Keemo Paul,-141.504136
Lasith Malinga,-71.679426
Moeen Ali,-122.420616


In [31]:
delta_min = min(delta_sums)
delta_max = max(delta_sums)
pd.DataFrame(data=[delta_min, delta_max], columns=['Value'], index=['Minimum', 'Maximum'])

,Value
Minimum,-210.295741
Maximum,-7.143317


In [32]:
ratings = (delta_sums - delta_min) / (delta_max - delta_min)
pd.DataFrame(data=ratings, index=candidates, columns=['Rating'])

,Rating
Andre Russell,0.636970
Ben Stokes,0.159404
Chris Morris,0.485163
Dwayne Bravo,0.578788
Imran Tahir,1.000000
Jofra Archer,0.597523
Kagiso Rabada,0.917569
Keemo Paul,0.338621
Lasith Malinga,0.682327
Moeen Ali,0.432558


## Step 4 - Create ranking based on the calculated $\zeta_i$ values

In [33]:
def rank_according_to(data):
    ranks = (rankdata(data) - 1).astype(int)
    storage = np.zeros_like(candidates)
    storage[ranks] = candidates
    return storage[::-1]

In [34]:
result = rank_according_to(ratings)
pd.DataFrame(data=result, index=range(1, m + 1), columns=['Name'])

,Name
1,Imran Tahir
2,Kagiso Rabada
3,Rashid Khan
4,Lasith Malinga
5,Andre Russell
6,Jofra Archer
7,Dwayne Bravo
8,Chris Morris
9,Sunil Narine
10,Mohammad Nabi


In [35]:
print("The best candidate/alternative according to C* is " + result[0])
print("The preferences in descending order are " + ", ".join(result) + ".")

The best candidate/alternative according to C* is Imran Tahir
The preferences in descending order are Imran Tahir, Kagiso Rabada, Rashid Khan, Lasith Malinga, Andre Russell, Jofra Archer, Dwayne Bravo, Chris Morris, Sunil Narine, Mohammad Nabi, Moeen Ali, Sam Curran, Keemo Paul, Ben Stokes, Trent Boult.
